In [88]:
import pandas as pd

# Load data
hits_df = pd.read_csv('../../resources/enterobacterales/test_blastp_blastn/results/final/hmm_hits_with_pident.csv')
ss_df = pd.read_csv('../../resources/enterobacterales/test_blastp_blastn/results/hmmsearch/ss_domains.csv')
gp_df = pd.read_csv('../../resources/enterobacterales/test_blastp_blastn/results/hmmsearch/ss_gram_positive_domains.csv')
microcin_domain_hmm_df = pd.read_csv('../../resources/enterobacterales/test_blastp_blastn/results/hmmsearch/extracted_hits_domain.csv')

#filter hits_df where column 'reg' is greater than 1
ss_df = ss_df[ss_df['evalue'] < 1]
gp_df = gp_df[gp_df['evalue'] < 1]
microcin_domain_hmm_df = microcin_domain_hmm_df[microcin_domain_hmm_df['evalue'] < 1]

#rename evalue column to evalue_signalsequence in ss_df
ss_df = ss_df.rename(columns={'evalue': 'evalue_signal_sequence'})
gp_df = gp_df.rename(columns={'evalue': 'evalue_gram_positive'})
microcin_domain_hmm_df = microcin_domain_hmm_df.rename(columns={'evalue': 'evalue_microcin_domain'})

#rename column 'hit_id' to 'pephash' in ss_df and gp_df
ss_df = ss_df.rename(columns={'hit_id': 'pephash'})
gp_df = gp_df.rename(columns={'hit_id': 'pephash'})
microcin_domain_hmm_df = microcin_domain_hmm_df.rename(columns={'hit_id': 'contig'})

#merge left hits_df and right hmm_dfs on pephash and target name
merged_df = pd.merge(hits_df, ss_df, how='left', on='pephash')
merged_df = pd.merge(merged_df, gp_df, how='left', on='pephash')
merged_df = pd.merge(merged_df, microcin_domain_hmm_df, how='left', on='contig')

#rename columns hit_start_x to hit_start_signalsequence and hit_start_y to hit_start_gram_positive
merged_df = merged_df.rename(columns={'hit_start_x': 'hit_start_signal_sequence', 'hit_start_y': 'hit_start_gram_positive', 'hit_start': 'hit_start_microcin_domain'})

#rename columns query_start_x to query_start_signalsequence and query_start_y to query_start_gram_positive
merged_df = merged_df.rename(columns={'query_start_x': 'query_start_signal_sequence', 'query_start_y': 'query_start_gram_positive', 'query_start': 'query_start_microcin_domain'})

#write csv merged_df
merged_df.to_csv('../../resources/enterobacterales/test_blastp_blastn/results/final/hmm_hits_with_domain_hits.csv', index=False)


In [89]:
import pandas as pd
import numpy as np

#load df from csv of file hmm_hits_with_ss_hit.csv
df = pd.read_csv('../../resources/enterobacterales/test_blastp_blastn/results/final/hmm_hits_with_domain_hits.csv')

#fill empty cells with nan
df = df.fillna(np.nan)

# Remove all rows that have dont have 'hit_start' and have an 'evalue' < 1
sub_df = df[(df['evalue'] < 1) | (df['hit_start_signal_sequence'].notna()) | (df['hit_start_gram_positive'].notna())]

# Shorten seq if signal sequence hmm matched
for index, row in sub_df.iterrows():
    if row.hit_start_signal_sequence > 0:
        position = int(row.hit_start_signal_sequence)+int(row.query_start_signal_sequence)
        while all(elem not in ['M'] for elem in row.seq[position]):
            position -= 1
        sub_df.at[index, 'seq'] = row.seq[position:]
        
# Shorten seq if gram positive hmm matched
for index, row in sub_df.iterrows():
    if row.hit_start_gram_positive > 0:
        position = int(row.hit_start_gram_positive)+int(row.query_start_gram_positive)
        while all(elem not in ['M'] for elem in row.seq[position]):
            position -= 1
        sub_df.at[index, 'seq'] = row.seq[position:]

# remove all rows with query_start_microcin_domain > 140
sub_df = sub_df[(sub_df['query_start_microcin_domain'] <= 5) | sub_df['query_start_microcin_domain'].isna()]

# Shorten seq with original full sequence HMM if neither gram+ or ss matched
for index, row in sub_df.iterrows():
    if row.hit_start_microcin_domain > 0:
        if not pd.notna(row.hit_start_signal_sequence) and not pd.notna(row.hit_start_gram_positive):
            position = int(row.hit_start_microcin_domain)+int(row.query_start_microcin_domain)
            while all(elem not in ['M'] for elem in row.seq[position]):
                position -= 1
            # Replace seq in sub_df with row.seq[position:]
            sub_df.at[index, 'seq'] = row.seq[position:]

# remove rows with length over 140
sub_df = sub_df[sub_df['seq'].str.len() <= 140]

# Write CSV
sub_df.to_csv('../../resources/enterobacterales/test_blastp_blastn/results/final/best_hits_short.csv', index=False)


In [76]:
# Check that a 'G' occurs in the 12-18th position of the sequence
#for index, row in sub_df.iterrows():
#    print(row.seq[12:18])
for index, row in sub_df.iterrows():
    substring_list = ["GG", "GA", "GT", "GS", "GD"]
    contains_substring = any(substring in row.seq[12:18] for substring in substring_list)
    print(contains_substring)
    print(row.seq[11:18])


True
VSGAGAP
True
ISGAGEF
True
VYGSVDN
True
VSGAGNV
True
IAAAGDF
True
VPGAGDV
True
ISGGDGG
True
VSGAGNY
True
MKLIGGC
True
ISGAGDR
True
ISGGRGG
True
VSGGDGG
True
DLISGGN
True
VSGGDSG
True
VSGGDSG
True
VSGGNAN
True
IYGSDGN
True
VYGGDGG
False
DSKAMED
False
QYTVNPN
True
TSGSDGN
True
VYGGDGG
True
IAGGRGN
True
ISGGNAD
True
ISGGNAD
False
RRLSALE
True
IGGGGES
True
VSGGDSG
True
VSGAGFF
True
VDGGGDG
True
ISGGWTA
True
VSGGDDF
True
NVSGGNA
True
VDGGGDG
True
VNGGGVV
False
LHNEREL
True
VSGAYGD
True
ASAAGTP
True
VSGAGVI
True
VAGGDGA
True
VSGAGFF
True
VSGGEFV
False
KSGHIFS
True
VSGAWTS
False
VVVTQDK
True
VNGGCIE
True
IAGAGNF
True
VSGAGGL
True
VTGASGA
False
EIAFISG
True
VDGGFGL
False
PYYWCNS
True
VQGGILP
True
VNGAGII
False
SEEIKTV
True
VSGGSFF
False
VSGCGLI
True
VTGGVAP
True
VNGAGFV
True
VSGGASS
True
VSGAGAL
True
DSIAGGR
False
TRKRALA
True
VSGGDYY
True
VSGAGLW
False
TRKRALA
True
VSGGDLN
True
VSGGDLN
False
KSEIDMI
False
VVVSKDK
True
ISGGREA
True
VSGAGKI
True
VSGAGLF
True
VNGGWAK
False
MKANALS
False
MKAN

In [55]:
from Bio import SearchIO

hit_attribs = ['id', 'bitscore', 'evalue', 'bias']
hsp_attribs = ['env_start', 'env_end']
hits = []

file_path = "../../resources/enterobacterales/test_blastp_blastn/results/hmmsearch/ss_hmmsearch_domtblout.txt"

results = SearchIO.parse(file_path, 'hmmscan3-domtab')

for result in results:
#    print(result.id)
    for hit in result:
#        print(hit.id)
        for hsp in hit:
            # print(hsp)
            print(hsp.hit_id)
            print(hsp.evalue)
            print(hsp.hit_start)
            print(hsp.query_start)


v1_PLS_43e35dac32e4e33535626414ebb39c62b8f533b825da21d2e543ef9db21d35c5
1.7e-06
0
0
v1_PLS_c7cf9039519ff486dba0f10aa4eff2f7daada6a2496366a29d391562fa6fa5da
2.4e-06
0
20
v1_PLS_dfaced24dfac66bf693b7011abd5471d8a280d6b793218895ed5498a82b57f4f
6.2e-06
0
0
v1_PLS_287df586b104fbf4f67cc43a9167a638a30b135b6befb420e4dcbc96e2b6186b
1.1e-05
0
4
v1_PLS_19a21776df0c06234ea03db35840d41fa816520e7319067065d0f82122c4d99e
8.4e-06
1
7
v1_PLS_3ccb58e107e7cdddfe931c43545c97f7dd3d7a7f77732b99859f5bbccd0430c6
1.1e-05
0
0
v1_PLS_b0651204ee63eb822d671edee6d5704293984b9d6736f80ed35690dafcba7fe9
1.7e-05
0
0
v1_PLS_5b6239dd42f797a40e6fcc9ca8571caa7a882d8d0aab5a95522844cece9a3e7a
1.5e-05
0
0
v1_PLS_5352c6d709f1a1f31a68f907b9750eed689cec581d26362fe565265ecd9ed519
1e-05
0
0
v1_PLS_6d588212416814332f2793aa6a8ec0ebfa560742c883ad5bc5b05e5b225136bf
1e-05
0
0
v1_PLS_fb3bf6c320b4281b7f45b31e06b363862ab4ca77a544b4df29cd246146d64ce9
1.3e-05
0
0
v1_PLS_cf121f399716d92d36e49c7a7eb89e77b7cdbeeb4a8d7973fed48c60067921e9
3.2e-05

In [3]:
df = pd.read_csv('../../resources/test_genome/test_new_args_4/results/hmmsearch/hmm_hits.csv')


FileNotFoundError: [Errno 2] No such file or directory: '../../resources/test_genome/test_new_args_4/results/hmmsearch/hmm_hits.csv'

In [20]:
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord


def pandas_df_to_fasta(dataframe, sequence_column, header_column, output_file):
    records = []

    for _, row in dataframe.iterrows():
        seq_id = row[header_column]
        sequence = row[sequence_column]
        sequence = Seq(sequence)
        record = SeqRecord(sequence, id=seq_id, description="")
        records.append(record)

    with open(output_file, "w") as output_handle:
        SeqIO.write(records, output_handle, "fasta")


In [21]:
df = pd.read_csv('../../resources/test_genome/test_new_args3/results/hmmsearch/hmm_hits.csv')
df = df[["pephash", "seq"]]
#Trim * from seq
df["seq"] = df["seq"].str.replace("*", "", regex=False)
df

pandas_df_to_fasta(df, 'seq', 'pephash', 'test_out.fa')

,pephash,seq
0,v1_PLS_30a7a42d3b5f6395cd721cca1240930eccfdd03...,MRELNSEEIRNVAGAGTDNPYDNPILLMNQIAENAAWGAALGAYGG...
1,v1_PLS_b7c39251a51415725e47af1f392d4c7495fbdad...,MRLLNAAELKTVSGADPGYSDLL
2,v1_PLS_b3dfdade24c080bc500e5d3e6a5e157c0cb60b8...,MSLVDYAAARQSVPSRRYRFAPGLWLATFVMLLAVLAALFPAIFTQ...


In [1]:
num_queries = 0
evalue = 10
biased_composition_filter = True

threads = int(90)
if threads > 2:
    cpu = threads - 1
else: 
    cpu = 1

# Generate string for running hmmsearch
#setup = f'"cpu=$(({threads} > 2 ? ({threads} - 1) : 1)) ; '
command = f'"hmmsearch -E {evalue} ' 
if num_queries == 0:
    Z = ''
else:
    Z = f'-Z {num_queries} '
if biased_composition_filter:
    bias = ''
else:
    bias = '--nobias '
T = f'--cpu {cpu} '
end = '--tblout {output.tblout} --domtblout {output.domtblout} {input.hmmfile} {input.seqdb} > {output.fa}"'

hmmsearch = command + Z + bias + T + end
hmmsearch

'"hmmsearch -E 10 --cpu 89 --tblout {output.tblout} --domtblout {output.domtblout} {input.hmmfile} {input.seqdb} > {output.fa}"'

In [43]:
import pandas as pd

#new shit
SAMPLES = ['GCA_002554555.1_PDEG01.1', 'GCA_002554555.1_PDEG01.1']
print('length of samples is ' + str(len(SAMPLES)))
# Create empty DataFrame
contig_to_genome_table = pd.DataFrame()
subset_contig_to_genome = pd.DataFrame(columns=['genome', 'contig'])
# Iterate through each sample
count = -1
for i in range(len(SAMPLES)):
    print("i is " + str(i))
    print('count is ' + str(count))
    # Assign sample to the current sample
    sample = SAMPLES[i]

    print("sample is " + sample)
    # Read ORF file and assign it to orfs_df
    orfs_df = pd.read_csv("../../resources/test_genome/Test_read_files_may30/results/ORFs/" + sample + "_ORFs.csv")
#    print(orfs_df['sample'])
    # Create a new column called 'sample' and assign it to the sample name
    for j in range(len(orfs_df['sample'].unique())):
        count += 1
        print(count)
        #make dataframe with two columns, 
        line = pd.DataFrame([[sample, orfs_df['sample'].unique()[j]]], columns=['genome', 'contig'])

        # Append orfs_df to contig_to_genome_table
        contig_to_genome_table = contig_to_genome_table.append(line)


print(contig_to_genome_table)

# # Write the final completed DataFrame to the output file
# print(contig_to_genome_table)


length of samples is 2
i is 0
count is -1
sample is GCA_002554555.1_PDEG01.1
0
1
2
3
4
i is 1
count is 4
sample is GCA_002554555.1_PDEG01.1


/tmp/ipykernel_68158/1245304807.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  contig_to_genome_table = contig_to_genome_table.append(line)
/tmp/ipykernel_68158/1245304807.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  contig_to_genome_table = contig_to_genome_table.append(line)
/tmp/ipykernel_68158/1245304807.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  contig_to_genome_table = contig_to_genome_table.append(line)
/tmp/ipykernel_68158/1245304807.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  contig_to_genome_table = contig_to_genome_table.append(line)
/tmp/ipykernel_68158/1245304807.py:29: FutureWarning: The fr

5
6
7
8
9
                     genome          contig
0  GCA_002554555.1_PDEG01.1  PDEG01000001.1
0  GCA_002554555.1_PDEG01.1  PDEG01000002.1
0  GCA_002554555.1_PDEG01.1  PDEG01000003.1
0  GCA_002554555.1_PDEG01.1  PDEG01000004.1
0  GCA_002554555.1_PDEG01.1  PDEG01000005.1
0  GCA_002554555.1_PDEG01.1  PDEG01000001.1
0  GCA_002554555.1_PDEG01.1  PDEG01000002.1
0  GCA_002554555.1_PDEG01.1  PDEG01000003.1
0  GCA_002554555.1_PDEG01.1  PDEG01000004.1
0  GCA_002554555.1_PDEG01.1  PDEG01000005.1


/tmp/ipykernel_68158/1245304807.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  contig_to_genome_table = contig_to_genome_table.append(line)
/tmp/ipykernel_68158/1245304807.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  contig_to_genome_table = contig_to_genome_table.append(line)
/tmp/ipykernel_68158/1245304807.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  contig_to_genome_table = contig_to_genome_table.append(line)
/tmp/ipykernel_68158/1245304807.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  contig_to_genome_table = contig_to_genome_table.append(line)
/tmp/ipykernel_68158/1245304807.py:29: FutureWarning: The fr

In [47]:
import pandas as pd

# New code
SAMPLES = ['GCA_002554555.1_PDEG01.1', 'GCA_002554555.1_PDEG01.1']
print('Length of samples is ' + str(len(SAMPLES)))

# Create empty DataFrame
contig_to_genome_table = pd.DataFrame()
subset_contig_to_genome = pd.DataFrame(columns=['genome', 'contig'])

# Iterate through each sample
count = -1
for i in range(len(SAMPLES)):    
    # Assign sample to the current sample
    sample = SAMPLES[i]
    
    # Read ORF file and assign it to orfs_df
    orfs_df = pd.read_csv("../../resources/test_genome/Test_read_files_may30/results/ORFs/" + sample + "_ORFs.csv")

    # Create a new column called 'sample' and assign it to the sample name
    for j in range(len(orfs_df['sample'].unique())):
        count += 1
        
        # Make DataFrame with two columns
        line = pd.DataFrame([[sample, orfs_df['sample'].unique()[j]]], columns=['genome', 'contig'])
        
        # Concatenate line with contig_to_genome_table
        contig_to_genome_table = pd.concat([contig_to_genome_table, line], ignore_index=True)
    

# Write the final completed DataFrame to the output file
print(contig_to_genome_table)

Length of samples is 2
                     genome          contig
0  GCA_002554555.1_PDEG01.1  PDEG01000001.1
1  GCA_002554555.1_PDEG01.1  PDEG01000002.1
2  GCA_002554555.1_PDEG01.1  PDEG01000003.1
3  GCA_002554555.1_PDEG01.1  PDEG01000004.1
4  GCA_002554555.1_PDEG01.1  PDEG01000005.1
5  GCA_002554555.1_PDEG01.1  PDEG01000001.1
6  GCA_002554555.1_PDEG01.1  PDEG01000002.1
7  GCA_002554555.1_PDEG01.1  PDEG01000003.1
8  GCA_002554555.1_PDEG01.1  PDEG01000004.1
9  GCA_002554555.1_PDEG01.1  PDEG01000005.1


In [81]:
# Load hits
extracted_df = pd.read_csv('../../resources/test_genome/Test_read_files_may30/results/hmmsearch/extracted_hits.csv').rename(columns={'id': 'contig'})

# Add blank columns
added_columns = ['id', 'pephash', 'sample', 'start', 'stop', 'strand', 'dna', 'seq']
for column in added_columns:
    extracted_df[column] = ''

contig_to_genome_table = pd.read_csv('../../resources/test_genome/Test_read_files_may30/results/ORFs/contig_to_genome_table.csv')

# Iterate through each row of extracted_df
for i in range(len(extracted_df)):
    # Find the genome that the contig belongs to
    print(extracted_df.loc[i, 'contig'].split('_')[0])
    print(contig_to_genome_table['contig'] == extracted_df.loc[i, 'contig'].split('_')[0])
    print(contig_to_genome_table[contig_to_genome_table['contig'] == extracted_df.loc[i, 'contig'].split('_')[0]])
#     print(genome_row)
# #    genome_row = contig_to_genome_table[contig_to_genome_table['contig'] == extracted_df.loc[i, 'contig']]
#     if not genome_row.empty:
#         genome = genome_row.iloc[0]['genome']

#         # Generate the string for ORF file for the genome found
#         orf_file = "../../resources/test_genome/Test_read_files_may30/results/ORFs/" + genome + "_ORFs.csv"

#         # Read ORF file and assign it to orfs_df
#         orfs_df = pd.read_csv(orf_file)
#         # Find the row in orfs_df that matches the 'contig' (should be only 1)
#         matching_row = orfs_df[orfs_df['contig'] == extracted_df.loc[i, 'contig']]
#         # Add values from columns to extracted_df
#         for column in added_columns:
#             extracted_df.loc[i, column] = matching_row.loc[0,column]

# # Write the final completed DataFrame to the output file
# print(extracted_df)

PDEG01000002.1
0    False
1     True
2    False
3    False
4    False
Name: contig, dtype: bool
                     genome          contig
1  GCA_002554555.1_PDEG01.1  PDEG01000002.1
PDEG01000001.1
0     True
1    False
2    False
3    False
4    False
Name: contig, dtype: bool
                     genome          contig
0  GCA_002554555.1_PDEG01.1  PDEG01000001.1
